In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using 

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"
new_model = "rust-llm"
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/content/drive/MyDrive/hugging_face/rust_llm"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 100

# Log every X updates steps
logging_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 512

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Prepare dataset

SYSTEM_PROMPT = "Write a full Rust script with a valid main entry point that solves the following problem.The program should run with all provided examples and output to stdout."
def generate_and_tokenize_prompt(data_point):
    full_prompt = f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT.strip()}\n<</SYS>>\n\n{data_point['instruction']} [/INST] ```{data_point['code']}``` </s>"
    return {'full_prompt': full_prompt}

def truncate_output(data_point):
  x = data_point['output'].split('```')
  if len(x) == 1 :
    return {'code': data_point['output']}
  else:
    return {'code': x[1].split('```')[0]}


dataset = load_dataset('Neloy262/rust_instruction_dataset',split='train')
dataset = dataset.map(truncate_output)
dataset = dataset.map(generate_and_tokenize_prompt)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="full_prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
trainer.train(resume_from_checkpoint = True)

Step,Training Loss
2300,0.368100
2400,0.342400
2500,0.358800


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=2500, training_loss=0.04277136688232422, metrics={'train_runtime': 3303.5892, 'train_samples_per_second': 3.027, 'train_steps_per_second': 0.757, 'total_flos': 9.332148994972877e+16, 'train_loss': 0.04277136688232422, 'epoch': 1.0})

In [ ]:
trainer.model.save_pretrained("/content/drive/MyDrive/hugging_face/final_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/hugging_face/final_model")
model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model.push_to_hub("rust_llm")
tokenizer.push_to_hub("rust_llm")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ripper479/rust_llm/commit/e3d031645383c277679aa284b0f78679aec8ccf3', commit_message='Upload tokenizer', commit_description='', oid='e3d031645383c277679aa284b0f78679aec8ccf3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#Inference

SYSTEM_PROMPT = "Write a full Rust script with a valid main entry point that solves the following problem.The program should run with all provided examples and output to stdout."
def generate_and_tokenize_prompt_inference(data_point):
    full_prompt = f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT.strip()}\n<</SYS>>\n\n{data_point['content']} [/INST]"
    return {'full_prompt': full_prompt}

In [ ]:
from datasets import load_dataset, Dataset

dataset = load_dataset("json", data_files="leetcode_problems.jsonl", split='train')

prompted_dataset = dataset.map(generate_and_tokenize_prompt_inference)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2360 [00:00<?, ? examples/s]

In [ ]:
prompt = prompted_dataset[100]['full_prompt']
print(len(prompt))
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

639


In [ ]:
prompted_dataset[100]

{'id': 101,
 'slug': 'symmetric-tree',
 'title': 'Symmetric Tree',
 'difficulty': 'Easy',
 'content': 'Given the `root` of a binary tree, _check whether it is a mirror of itself_ (i.e., symmetric around its center).\n\n**Example 1:**\n\n**Input:** root = \\[1,2,2,3,4,4,3\\]\n**Output:** true\n\n**Example 2:**\n\n**Input:** root = \\[1,2,2,null,3,null,3\\]\n**Output:** false\n\n**Constraints:**\n\n*   The number of nodes in the tree is in the range `[1, 1000]`.\n*   `-100 <= Node.val <= 100`\n\n**Follow up:** Could you solve it both recursively and iteratively?',
 'java': "\n    ```java\npublic boolean isSymmetric(TreeNode root) {\n    return checkSymmetry(root, root);\n}\n\npublic boolean checkSymmetry(TreeNode node1, TreeNode node2) {\n    if (node1 == null && node2 == null) return true;\n    if (node1 == null || node2 == null) return false;\n    return (node1.val == node2.val) && checkSymmetry(node1.right, node2.left) && checkSymmetry(node1.left, node2.right);\n}\n```\nThe algorithm 

In [ ]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> [INST] <<SYS>>
Write a full Rust script with a valid main entry point that solves the following problem.The program should run with all provided examples and output to stdout.
<</SYS>>

Given the `root` of a binary tree, _check whether it is a mirror of itself_ (i.e., symmetric around its center).

**Example 1:**

**Input:** root = \[1,2,2,3,4,4,3\]
**Output:** true

**Example 2:**

**Input:** root = \[1,2,2,null,3,null,3\]
**Output:** false

**Constraints:**

*   The number of nodes in the tree is in the range `[1, 1000]`.
*   `-100 <= Node.val <= 100`

**Follow up:** Could you solve it both recursively and iteratively? [/INST] ```rust
// Definition for a binary tree node.
#[derive(Debug, PartialEq, Eq)]
pub struct TreeNode {
    pub val: i32,
    pub left: Option<Box<TreeNode>>,
    pub right: Option<Box<TreeNode>>,
}

impl TreeNode {
    pub fn new(val: i32) -> Self {
        TreeNode {
            val,
            left: None,
            right: None,
        }
    }
}

pub s

In [ ]:
# Inference Leetcode problems
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = "Write a full Rust script with a valid main entry point that solves the following problem.The program should run with all provided examples and output to stdout."
def generate_and_tokenize_prompt_inference(data_point):
    full_prompt = f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT.strip()}\n<</SYS>>\n\n{data_point['content']} [/INST]"
    return {'full_prompt': full_prompt}

dataset = load_dataset("json", data_files="leetcode_problems.jsonl", split='train')

prompted_dataset = dataset.map(generate_and_tokenize_prompt_inference)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2360 [00:00<?, ? examples/s]

In [ ]:
# Inference Leetcode problems
from datasets import load_dataset, Dataset


def generate_and_tokenize_prompt_inference(data_point):
    full_prompt = f"<s>[INST] <<SYS>>\nWrite a Rust function with the following function signature {data_point['function']} that solves the following problem.The program should run with all provided examples and output to stdout.\n<</SYS>>\n\n{data_point['content']} [/INST]"
    return {'full_prompt': full_prompt}

dataset = load_dataset("ripper479/leetcode_150",split='train')

prompted_dataset = dataset.map(generate_and_tokenize_prompt_inference)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
print(prompted_dataset[0]['full_prompt'])

<s>[INST] <<SYS>>
Write a Rust function with the following function signature impl Solution {     pub fn merge(nums1: &mut Vec<i32>, m: i32, nums2: &mut Vec<i32>, n: i32) {              } } that solves the following problem.The program should run with all provided examples and output to stdout.
<</SYS>>

You are given two integer arrays `nums1` and `nums2`, sorted in **non-decreasing order**, and two integers `m` and `n`, representing the number of elements in `nums1` and `nums2` respectively.

**Merge** `nums1` and `nums2` into a single array sorted in **non-decreasing order**.

The final sorted array should not be returned by the function, but instead be _stored inside the array_ `nums1`. To accommodate this, `nums1` has a length of `m + n`, where the first `m` elements denote the elements that should be merged, and the last `n` elements are set to `0` and should be ignored. `nums2` has a length of `n`.

**Example 1:**

**Input:** nums1 = \[1,2,3,0,0,0\], m = 3, nums2 = \[2,5,6\], n 

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model_tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", trust_remote_code=True)

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "ripper479/rust_llm",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model_tokenizer = AutoTokenizer.from_pretrained("ripper479/rust_llm", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.85k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

In [ ]:
from tqdm.notebook import tqdm

model.eval()
responses = []
with torch.no_grad():
  for i in tqdm(range(150)):
    prompt = prompted_dataset[i]['full_prompt']
    model_input = model_tokenizer(prompt, return_tensors="pt").to("cuda")
    response = model_tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0])
    responses.append(response)

  0%|          | 0/150 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [ ]:
import numpy as np
np.save('codellama.npy', responses)

In [ ]:
print(responses[51])

<s><s> [INST] <<SYS>>
Write a full Rust script with a valid main entry point that solves the following problem.The program should run with all provided examples and output to stdout.
<</SYS>>

The **n-queens** puzzle is the problem of placing `n` queens on an `n x n` chessboard such that no two queens attack each other.

Given an integer `n`, return _the number of distinct solutions to the **n-queens puzzle**_.

**Example 1:**

**Input:** n = 4
**Output:** 2
**Explanation:** There are two distinct solutions to the 4-queens puzzle as shown.

**Example 2:**

**Input:** n = 1
**Output:** 1

**Constraints:**

*   `1 <= n <= 9` [/INST] ```rust
pub fn total_n_queens(n: usize) -> usize {
    let mut dp = vec![1; n];
    for i in 1..n {
        for j in 0..i {
            if i - j != abs(i - j) {
                dp[i] += dp[j];
            }
        }
    }
    dp[n - 1]
}

fn abs(x: usize) -> usize {
    x.max(x.wrapping_neg())
}
``` 
This solution uses dynamic programming to calculate the nu

In [3]:
import numpy as np
x = np.load('codellama.npy')
y = np.load('rust_llm.npy')
dataset = load_dataset("ripper479/leetcode_150",split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [106]:
print(dataset[149]['title'])

Maximal Square


In [105]:
print(x[149])

<s><s> [INST] <<SYS>>
Write a Rust function with the following function signature impl Solution {     pub fn maximal_square(matrix: Vec<Vec<char>>) -> i32 {              } } that solves the following problem.The program should run with all provided examples and output to stdout.
<</SYS>>

Given an `m x n` binary `matrix` filled with `0`'s and `1`'s, _find the largest square containing only_ `1`'s _and return its area_.

**Example 1:**

**Input:** matrix = \[\[ "1 ", "0 ", "1 ", "0 ", "0 "\],\[ "1 ", "0 ", "1 ", "1 ", "1 "\],\[ "1 ", "1 ", "1 ", "1 ", "1 "\],\[ "1 ", "0 ", "0 ", "1 ", "0 "\]\]
**Output:** 4

**Example 2:**

**Input:** matrix = \[\[ "0 ", "1 "\],\[ "1 ", "0 "\]\]
**Output:** 1

**Example 3:**

**Input:** matrix = \[\[ "0 "\]\]
**Output:** 0

**Constraints:**

*   `m == matrix.length`
*   `n == matrix[i].length`
*   `1 <= m, n <= 300`
*   `matrix[i][j]` is `'0'` or `'1'`. [/INST]  Here is a possible implementation of the `maximal_square` function:
```
impl Solution {
    

In [104]:
print(y[149])

<s><s> [INST] <<SYS>>
Write a Rust function with the following function signature impl Solution {     pub fn maximal_square(matrix: Vec<Vec<char>>) -> i32 {              } } that solves the following problem.The program should run with all provided examples and output to stdout.
<</SYS>>

Given an `m x n` binary `matrix` filled with `0`'s and `1`'s, _find the largest square containing only_ `1`'s _and return its area_.

**Example 1:**

**Input:** matrix = \[\[ "1 ", "0 ", "1 ", "0 ", "0 "\],\[ "1 ", "0 ", "1 ", "1 ", "1 "\],\[ "1 ", "1 ", "1 ", "1 ", "1 "\],\[ "1 ", "0 ", "0 ", "1 ", "0 "\]\]
**Output:** 4

**Example 2:**

**Input:** matrix = \[\[ "0 ", "1 "\],\[ "1 ", "0 "\]\]
**Output:** 1

**Example 3:**

**Input:** matrix = \[\[ "0 "\]\]
**Output:** 0

**Constraints:**

*   `m == matrix.length`
*   `n == matrix[i].length`
*   `1 <= m, n <= 300`
*   `matrix[i][j]` is `'0'` or `'1'`. [/INST] ```rust
pub fn maximal_square(matrix: Vec<Vec<char>>) -> i32 {
    let m = matrix.len();
    

In [ ]:
z = '''impl Solution {
    pub fn merge(nums1: &mut Vec<i32>, m: i32, nums2: &mut Vec<i32>, n: i32) {

    }
}'''

In [ ]:
z

'impl Solution {\n    pub fn merge(nums1: &mut Vec<i32>, m: i32, nums2: &mut Vec<i32>, n: i32) {\n        \n    }\n}'

In [ ]:
func_struct = []
for i in range(150):
  id = input("Enter id of the problem")
  title = input("Enter the title of the problem: ")
  func = input("Enter the function structure of the problem: ")
  func_struct.append({"id": id, "title": title, "function": func})

Enter id of the problem88
Enter the title of the problem: Merge Sorted Array
Enter the function structure of the problem: impl Solution {     pub fn merge(nums1: &mut Vec<i32>, m: i32, nums2: &mut Vec<i32>, n: i32) {              } }
Enter id of the problem27
Enter the title of the problem: Remove Element
Enter the function structure of the problem: impl Solution {     pub fn remove_element(nums: &mut Vec<i32>, val: i32) -> i32 {              } }
Enter id of the problem26
Enter the title of the problem: Remove Duplicates from Sorted Array
Enter the function structure of the problem: impl Solution {     pub fn remove_duplicates(nums: &mut Vec<i32>) -> i32 {              } }
Enter id of the problem80
Enter the title of the problem: Remove Duplicates from Sorted Array II
Enter the function structure of the problem: impl Solution {     pub fn remove_duplicates(nums: &mut Vec<i32>) -> i32 {              } }
Enter id of the problem169
Enter the title of the problem: Majority Element
Enter th

In [ ]:
import numpy as np
# np.save('leetcode150.npy',func_struct)

In [ ]:
print(func_struct[149]['function'])

impl Solution {     pub fn maximal_square(matrix: Vec<Vec<char>>) -> i32 {              } }


In [3]:
lc150 = np.load('leetcode150.npy',allow_pickle=True)

NameError: name 'np' is not defined

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="leetcode_problems.jsonl", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def find_id(id):
  for i in range(len(dataset)):
    if dataset[i]['id'] == id:
      return i
  return -1


indexes = []
for i in range(150):
  id = int(lc150[i]['id'])
  index = find_id(id)
  indexes.append(index)

In [ ]:
problems = []
for i in range(150):
  problem_dict = {}
  problem_dict['id'] = lc150[i]['id']
  problem_dict['title'] = lc150[i]['title']
  problem_dict['function'] = lc150[i]['function']
  problem_dict['content'] = dataset[indexes[i]]['content']
  problems.append(problem_dict)

In [ ]:
import datasets
leetcode_150 = datasets.Dataset.from_list(problems)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
leetcode_150.push_to_hub("leetcode_150")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ripper479/leetcode_150/commit/72b26d2c164d8f01be5002d8df742b7a3b9c31fb', commit_message='Upload dataset', commit_description='', oid='72b26d2c164d8f01be5002d8df742b7a3b9c31fb', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
lc150 = load_dataset("ripper479/leetcode_150",split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [54]:
leetcode150 = {}
for i in range(150):
  id = lc150[i]['id']
  data = {}
  data['title'] = lc150[i]['title']
  data['content'] = lc150[i]['content']
  leetcode150[id] = data

In [5]:
import numpy as np
rust_responses = np.load('rust_llm.npy')
codellama_responses = np.load('codellama.npy')

In [16]:
rust_responses[0]

'<s><s> [INST] <<SYS>>\nWrite a Rust function with the following function signature impl Solution {     pub fn merge(nums1: &mut Vec<i32>, m: i32, nums2: &mut Vec<i32>, n: i32) {              } } that solves the following problem.The program should run with all provided examples and output to stdout.\n<</SYS>>\n\nYou are given two integer arrays `nums1` and `nums2`, sorted in **non-decreasing order**, and two integers `m` and `n`, representing the number of elements in `nums1` and `nums2` respectively.\n\n**Merge** `nums1` and `nums2` into a single array sorted in **non-decreasing order**.\n\nThe final sorted array should not be returned by the function, but instead be _stored inside the array_ `nums1`. To accommodate this, `nums1` has a length of `m + n`, where the first `m` elements denote the elements that should be merged, and the last `n` elements are set to `0` and should be ignored. `nums2` has a length of `n`.\n\n**Example 1:**\n\n**Input:** nums1 = \\[1,2,3,0,0,0\\], m = 3, nu

In [6]:
import pandas as pd
verdict = pd.read_csv('Leetcode_150_LLM_Results - Sheet1.csv')

In [22]:
verdict.iloc[1]

Problems                                     Merge Sorted Array
Unnamed: 1                                                  NaN
Rust_LLM                                      Compilation Error
Reason        Line 5: Char 21: error: mismatched types (solu...
Codellama                                     Compilation Error
Reason.1      Line 8: Char 22: error: the type `[i32]` canno...
Name: 1, dtype: object

In [7]:
def get_llm_response(s):
  x = s.split('```')
  if len(x) == 1:
    return x[0].split('[PYTHON]')[1].split('[/PYTHON]')[0]
  else:
    return x[1]

In [9]:
from tqdm.notebook import tqdm

finetuned_code = {}
finetuned_eval = {}
for i in tqdm(range(150)):
  key = str(lc150[i]['id'])
  data = {}
  llm_response = get_llm_response(rust_responses[i])
  finetuned_code[key] = llm_response
  if (verdict.iloc[i+1]['Rust_LLM'] == 'Compilation Error' or verdict.iloc[i+1]['Rust_LLM'] == 'No Rust Compiler' or verdict.iloc[i+1]['Rust_LLM'] == 'Context-Length Exceeded'):
    data['compiled'] = "false"
  else:
    data['compiled'] = "true"
  data['verdict'] = verdict.iloc[i+1]['Rust_LLM']
  if (verdict.iloc[i+1]['Rust_LLM'] == 'Compilation Error' or verdict.iloc[i+1]['Rust_LLM'] == 'Runtime Error'):
    data['stderr'] = verdict.iloc[i+1]['Reason']
  else:
    data['stderr'] = verdict.iloc[i+1]['Rust_LLM']
  finetuned_eval[key] = data


  0%|          | 0/150 [00:00<?, ?it/s]

In [14]:
from tqdm.notebook import tqdm

baseline_code = {}
baseline_eval = {}
for i in tqdm(range(150)):
  key = str(lc150[i]['id'])
  data = {}
  llm_response = get_llm_response(codellama_responses[i])
  baseline_code[key] = llm_response
  if (verdict.iloc[i+1]['Codellama'] == 'Compilation Error' or verdict.iloc[i+1]['Codellama'] == 'No Rust Compiler' or verdict.iloc[i+1]['Codellama'] == 'Context-Length Exceeded'):
    data['compiled'] = "false"
  else:
    data['compiled'] = "true"
  data['verdict'] = verdict.iloc[i+1]['Codellama']
  if (verdict.iloc[i+1]['Codellama'] == 'Compilation Error' or verdict.iloc[i+1]['Codellama'] == 'Runtime Error'):
    data['stderr'] = verdict.iloc[i+1]['Reason.1']
  else:
    data['stderr'] = verdict.iloc[i+1]['Codellama']
  baseline_eval[key] = data


  0%|          | 0/150 [00:00<?, ?it/s]

In [18]:
import json
with open('baseline_eval.json', 'w') as f:
    json.dump(baseline_eval, f)

In [38]:
lc150[52]['id']

'71'

In [16]:
baseline_code['71']

'\ndef simplify_path(path: str) -> str:\n    tokens = path.split("/")\n    stack = []\n    for token in tokens:\n        if token == "..":\n            if stack:\n                stack.pop()\n        elif token != ".":\n            stack.append(token)\n    return "/" + "/".join(stack)\n'